# GCC118 - Programação Matemática
## Universidade Federal de Lavras
### Instituto de Ciências Exatas e Tecnológicas
#### Profa. Andreza C. Beezão Moreira (DMM/UFLA)
#### Prof. Mayron César O. Moreira (DCC/UFLA)

Seu código deve ser testado na seguinte instância: [link](https://drive.google.com/file/d/12CeZEow-6vVgJFgzXMo0gbjV5hLCM6Zi/view?usp=sharing). O README se encontra em: [link](https://drive.google.com/file/d/1LBTdkDoTQRxUJsKLI4Z38-Uc8oUPCZQ0/view?usp=sharing).

## Alunos

| Nome                        | Matrícula |
|-----------------------------|-----------|
| Luiz Filipe Bartelega Penha | 202111082 |
| Vitor Pires Zini            | 202110169 |


### Instalando o Gurobi

In [1]:
!pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.4/14.4 MB 47.8 MB/s eta 0:00:00


In [6]:
from google.colab import files
import gurobipy as grb
import pandas as pd
import time

# Função para calcular o limite superior utilizando mochila fracionária (algoritmo guloso)
def limite_superior_guloso(itens, capacidade):
    # Ordena os itens pela razão valor/peso em ordem decrescente
    itens = sorted(itens, key=lambda x: x[1] / x[0], reverse=True)
    valor_total = 0
    peso_total = 0
    for peso, valor in itens:
        if peso_total + peso <= capacidade:
            valor_total += valor
            peso_total += peso
        else:
            capacidade_restante = capacidade - peso_total
            valor_total += valor * (capacidade_restante / peso)
            break
    return valor_total

# Função para calcular o limite inferior (heurística gulosa)
def heuristica_gulosa(itens, capacidade):
    itens = sorted(itens, key=lambda x: x[1] / x[0], reverse=True)
    valor_total = 0
    peso_total = 0
    for peso, valor in itens:
        if peso_total + peso <= capacidade:
            valor_total += valor
            peso_total += peso
        else:
            break
    return valor_total

# Branch and Bound para o problema da mochila binária
def branch_and_bound_mochila(itens, capacidade):
    n = len(itens)
    melhor_valor = 0
    melhor_solucao = None
    nos_explorados = 0

    # Inicializa o modelo Gurobi
    modelo = grb.Model()

    # Variáveis de decisão (0 ou 1 para cada item)
    x = modelo.addVars(n, vtype=grb.GRB.BINARY, name="x")

    # Função objetivo (maximizar o valor)
    modelo.setObjective(grb.quicksum(itens[i][1] * x[i] for i in range(n)), grb.GRB.MAXIMIZE)

    # Restrição de capacidade
    modelo.addConstr(grb.quicksum(itens[i][0] * x[i] for i in range(n)) <= capacidade, "Capacidade")

    # Resolver o problema
    modelo.optimize()

    if modelo.status == grb.GRB.OPTIMAL:
        melhor_valor = modelo.objVal
        melhor_solucao = [x[i].x for i in range(n)]

    return melhor_valor, melhor_solucao, nos_explorados

# Definição dos itens (peso, valor) e capacidade da mochila
uploaded = files.upload()
nome_arquivo = list(uploaded.keys())[0]
arquivo = pd.read_csv(nome_arquivo, sep='\s+', header=None, names=['Coluna1', 'Coluna2'])

capacidade = int(arquivo.loc[2, 'Coluna1'])

pesos = []
valores = []
itens = []

for peso in arquivo.iloc[3:, 0]:  # Começa da quarta linha e primeira coluna (pesos)
  pesos.append(int(peso))

i = 0
for valor in arquivo.iloc[3:, 1]:  # Começa da quarta linha e segunda coluna (valores)
  valores.append(int(valor))
  itens.append((pesos[0],valor))
  i += 1

# Executar o Branch and Bound
melhor_valor, melhor_solucao, nos_explorados = branch_and_bound_mochila(itens, capacidade)

# Calcular o GAP de otimalidade
valor_otimo = limite_superior_guloso(itens, capacidade)
gap = (valor_otimo - melhor_valor) / valor_otimo * 100

print(f"Melhor valor encontrado: {melhor_valor}")
print(f"Solução ótima: {melhor_solucao}")
print(f"Número de nós explorados: {nos_explorados}")
print(f"GAP de otimalidade: {gap:.2f}%")

Saving Weakly001 to Weakly001 (8)
Restricted license - for non-production use only - expires 2026-11-23
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 1 rows, 1000 columns and 1000 nonzeros
Model fingerprint: 0xcf9c2632
Variable types: 0 continuous, 1000 integer (1000 binary)
Coefficient statistics:
  Matrix range     [6e+00, 6e+00]
  Objective range  [3e+00, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+03, 2e+03]
Found heuristic solution: objective 205004.00000
Presolve removed 0 rows and 355 columns
Presolve time: 0.02s
Presolved: 1 rows, 645 columns, 645 nonzeros
Variable types: 0 continuous, 645 integer (392 binary)

Root relaxation: objective 3.246640e+05, 1 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bound